# Initial Modeling

## Setup

In [212]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from scipy import stats

from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, PowerTransformer, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, BaseCrossValidator
from scipy.stats import randint, uniform
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, make_scorer

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor, BaggingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

pd.set_option("display.max_columns", None)

## Data Prep

### Initial Look

In [213]:
og_df = pd.read_csv("./marts.csv")
og_df.columns = [col.lower() for col in og_df.columns]
og_df.sort_values("fix_id", inplace= True)
og_df.reset_index(drop= True, inplace= True)
df = og_df.drop(columns= ["label_goals", "label_assists"]).copy()
labels = og_df[["label_goals", "label_assists"]].copy()
og_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13137 entries, 0 to 13136
Columns: 132 entries, label_goals to rolling_goals_conceded_ub
dtypes: float64(106), int64(25), object(1)
memory usage: 13.2+ MB


In [214]:
df.head()

,player_id,fix_id,season,game_week,team_id,team_strength,opponent_strength,rolling_ub_minutes,rolling_ub_np_expected_goals,rolling_ub_expected_goals,rolling_ub_goals,rolling_ub_expected_assists,rolling_ub_assists,rolling_ub_shots,rolling_ub_shots_on_target,rolling_ub_pk_made,rolling_ub_pk_attempt,rolling_ub_yellow_cards,rolling_ub_red_cards,rolling_ub_touches,rolling_ub_tackles,rolling_ub_interceptions,rolling_ub_blocks,rolling_ub_shot_creating_actions,rolling_ub_goal_creating_actions,rolling_ub_pass_attempted,rolling_ub_pass_completed,rolling_ub_pass_progressive,rolling_ub_carries,rolling_ub_carries_progressive,rolling_ub_take_ons_attempted,rolling_ub_take_ons_success,rolling_bound_last3_minutes,rolling_bound_last3_expected_goals,rolling_bound_last3_np_expected_goals,rolling_bound_last3_expected_assists,rolling_bound_last3_goals,rolling_bound_last3_assists,rolling_bound_last3_shots,rolling_bound_last3_pk_made,rolling_bound_last3_pk_attempt,rolling_bound_last3_shots_on_target,rolling_bound_last3_yellow_cards,rolling_bound_last3_red_cards,rolling_bound_last3_touches,rolling_bound_last3_tackles,rolling_bound_last3_interceptions,rolling_bound_last3_blocks,rolling_bound_last3_shot_creating_actions,rolling_bound_last3_goal_creating_actions,rolling_bound_last3_pass_completed,rolling_bound_last3_pass_attempted,rolling_bound_last3_pass_progressive,rolling_bound_last3_carries,rolling_bound_last3_carries_progressive,rolling_bound_last3_take_ons_attempted,rolling_bound_last3_take_ons_success,rolling_bound_last6_minutes,rolling_bound_last6_expected_goals,rolling_bound_last6_np_expected_goals,rolling_bound_last6_expected_assists,rolling_bound_last6_goals,rolling_bound_last6_assists,rolling_bound_last6_shots,rolling_bound_last6_pk_made,rolling_bound_last6_pk_attempt,rolling_bound_last6_shots_on_target,rolling_bound_last6_yellow_cards,rolling_bound_last6_red_cards,rolling_bound_last6_touches,rolling_bound_last6_tackles,rolling_bound_last6_interceptions,rolling_bound_last6_blocks,rolling_bound_last6_shot_creating_actions,rolling_bound_last6_goal_creating_actions,rolling_bound_last6_pass_completed,rolling_bound_last6_pass_attempted,rolling_bound_last6_pass_progressive,rolling_bound_last6_carries,rolling_bound_last6_carries_progressive,rolling_bound_last6_take_ons_attempted,rolling_bound_last6_take_ons_success,goals_efficiency_ub,assists_efficiency_ub,shots_efficiency_ub,contribution_real_ub,contribution_xgi_ub,contribution_npxgi_ub,contribution_efficiency_xgi_ub,contribution_efficiency_npxgi_ub,goals_efficiency_last3,assists_efficiency_last3,shots_efficiency_last3,contribution_real_last3,contribution_xgi_last3,contribution_npxgi_last3,contribution_efficiency_xgi_last3,contribution_efficiency_npxgi_last3,goals_per_90_ub,assists_per_90_ub,real_contribution_per_90_ub,xg_per_90_ub,npxg_per_90_ub,xa_per_90_ub,xgi_per_90_ub,npxgi_per_90_ub,shots_per_90_ub,shots_on_target_per_90_ub,shot_creating_actions_per_90_ub,goal_creating_actions_per_90_ub,goals_per_90_last3,assists_per_90_last3,real_contribution_per_90_last3,xg_per_90_last3,npxg_per_90_last3,xa_per_90_last3,xgi_per_90_last3,npxgi_per_90_last3,shots_per_90_last3,shots_on_target_per_90_last3,shot_creating_actions_per_90_last3,goal_creating_actions_per_90_last3,team_goals_scored,team_goals_conceded,rolling_goals_scored_last3,rolling_goals_conceded_last3,rolling_goals_scored_last6,rolling_goals_conceded_last6,rolling_goals_scored_ub,rolling_goals_conceded_ub
0,1005,15,2022-23,2,7,2330,2520,90,NaN,NaN,0,NaN,0,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,1,0,1,0,1,0
1,1355,15,2022-23,2,7,2330,2520,90,0.0,0.0,0,0.0,0,1.0,0.

In [215]:
cols_to_drop = []

#### Numerical Columns

In [216]:
num_df = df.select_dtypes(include= np.number)
num_df.head()

,player_id,fix_id,game_week,team_id,team_strength,opponent_strength,rolling_ub_minutes,rolling_ub_np_expected_goals,rolling_ub_expected_goals,rolling_ub_goals,rolling_ub_expected_assists,rolling_ub_assists,rolling_ub_shots,rolling_ub_shots_on_target,rolling_ub_pk_made,rolling_ub_pk_attempt,rolling_ub_yellow_cards,rolling_ub_red_cards,rolling_ub_touches,rolling_ub_tackles,rolling_ub_interceptions,rolling_ub_blocks,rolling_ub_shot_creating_actions,rolling_ub_goal_creating_actions,rolling_ub_pass_attempted,rolling_ub_pass_completed,rolling_ub_pass_progressive,rolling_ub_carries,rolling_ub_carries_progressive,rolling_ub_take_ons_attempted,rolling_ub_take_ons_success,rolling_bound_last3_minutes,rolling_bound_last3_expected_goals,rolling_bound_last3_np_expected_goals,rolling_bound_last3_expected_assists,rolling_bound_last3_goals,rolling_bound_last3_assists,rolling_bound_last3_shots,rolling_bound_last3_pk_made,rolling_bound_last3_pk_attempt,rolling_bound_last3_shots_on_target,rolling_bound_last3_yellow_cards,rolling_bound_last3_red_cards,rolling_bound_last3_touches,rolling_bound_last3_tackles,rolling_bound_last3_interceptions,rolling_bound_last3_blocks,rolling_bound_last3_shot_creating_actions,rolling_bound_last3_goal_creating_actions,rolling_bound_last3_pass_completed,rolling_bound_last3_pass_attempted,rolling_bound_last3_pass_progressive,rolling_bound_last3_carries,rolling_bound_last3_carries_progressive,rolling_bound_last3_take_ons_attempted,rolling_bound_last3_take_ons_success,rolling_bound_last6_minutes,rolling_bound_last6_expected_goals,rolling_bound_last6_np_expected_goals,rolling_bound_last6_expected_assists,rolling_bound_last6_goals,rolling_bound_last6_assists,rolling_bound_last6_shots,rolling_bound_last6_pk_made,rolling_bound_last6_pk_attempt,rolling_bound_last6_shots_on_target,rolling_bound_last6_yellow_cards,rolling_bound_last6_red_cards,rolling_bound_last6_touches,rolling_bound_last6_tackles,rolling_bound_last6_interceptions,rolling_bound_last6_blocks,rolling_bound_last6_shot_creating_actions,rolling_bound_last6_goal_creating_actions,rolling_bound_last6_pass_completed,rolling_bound_last6_pass_attempted,rolling_bound_last6_pass_progressive,rolling_bound_last6_carries,rolling_bound_last6_carries_progressive,rolling_bound_last6_take_ons_attempted,rolling_bound_last6_take_ons_success,goals_efficiency_ub,assists_efficiency_ub,shots_efficiency_ub,contribution_real_ub,contribution_xgi_ub,contribution_npxgi_ub,contribution_efficiency_xgi_ub,contribution_efficiency_npxgi_ub,goals_efficiency_last3,assists_efficiency_last3,shots_efficiency_last3,contribution_real_last3,contribution_xgi_last3,contribution_npxgi_last3,contribution_efficiency_xgi_last3,contribution_efficiency_npxgi_last3,goals_per_90_ub,assists_per_90_ub,real_contribution_per_90_ub,xg_per_90_ub,npxg_per_90_ub,xa_per_90_ub,xgi_per_90_ub,npxgi_per_90_ub,shots_per_90_ub,shots_on_target_per_90_ub,shot_creating_actions_per_90_ub,goal_creating_actions_per_90_ub,goals_per_90_last3,assists_per_90_last3,real_contribution_per_90_last3,xg_per_90_last3,npxg_per_90_last3,xa_per_90_last3,xgi_per_90_last3,npxgi_per_90_last3,shots_per_90_last3,shots_on_target_per_90_last3,shot_creating_actions_per_90_last3,goal_creating_actions_per_90_last3,team_goals_scored,team_goals_conceded,rolling_goals_scored_last3,rolling_goals_conceded_last3,rolling_goals_scored_last6,rolling_goals_conceded_last6,rolling_goals_scored_ub,rolling_goals_conceded_ub
0,1005,15,2,7,2330,2520,90,NaN,NaN,0,NaN,0,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,1,0,1,0,1,0
1,1355,15,2,7,2330,2520,90,0.0,0.0,0,0.0,0,1.0,0.0,0,0,0,0,98.0,2.0,5.0,

In [217]:
num_df.describe()

,player_id,fix_id,game_week,team_id,team_strength,opponent_strength,rolling_ub_minutes,rolling_ub_np_expected_goals,rolling_ub_expected_goals,rolling_ub_goals,rolling_ub_expected_assists,rolling_ub_assists,rolling_ub_shots,rolling_ub_shots_on_target,rolling_ub_pk_made,rolling_ub_pk_attempt,rolling_ub_yellow_cards,rolling_ub_red_cards,rolling_ub_touches,rolling_ub_tackles,rolling_ub_interceptions,rolling_ub_blocks,rolling_ub_shot_creating_actions,rolling_ub_goal_creating_actions,rolling_ub_pass_attempted,rolling_ub_pass_completed,rolling_ub_pass_progressive,rolling_ub_carries,rolling_ub_carries_progressive,rolling_ub_take_ons_attempted,rolling_ub_take_ons_success,rolling_bound_last3_minutes,rolling_bound_last3_expected_goals,rolling_bound_last3_np_expected_goals,rolling_bound_last3_expected_assists,rolling_bound_last3_goals,rolling_bound_last3_assists,rolling_bound_last3_shots,rolling_bound_last3_pk_made,rolling_bound_last3_pk_attempt,rolling_bound_last3_shots_on_target,rolling_bound_last3_yellow_cards,rolling_bound_last3_red_cards,rolling_bound_last3_touches,rolling_bound_last3_tackles,rolling_bound_last3_interceptions,rolling_bound_last3_blocks,rolling_bound_last3_shot_creating_actions,rolling_bound_last3_goal_creating_actions,rolling_bound_last3_pass_completed,rolling_bound_last3_pass_attempted,rolling_bound_last3_pass_progressive,rolling_bound_last3_carries,rolling_bound_last3_carries_progressive,rolling_bound_last3_take_ons_attempted,rolling_bound_last3_take_ons_success,rolling_bound_last6_minutes,rolling_bound_last6_expected_goals,rolling_bound_last6_np_expected_goals,rolling_bound_last6_expected_assists,rolling_bound_last6_goals,rolling_bound_last6_assists,rolling_bound_last6_shots,rolling_bound_last6_pk_made,rolling_bound_last6_pk_attempt,rolling_bound_last6_shots_on_target,rolling_bound_last6_yellow_cards,rolling_bound_last6_red_cards,rolling_bound_last6_touches,rolling_bound_last6_tackles,rolling_bound_last6_interceptions,rolling_bound_last6_blocks,rolling_bound_last6_shot_creating_actions,rolling_bound_last6_goal_creating_actions,rolling_bound_last6_pass_completed,rolling_bound_last6_pass_attempted,rolling_bound_last6_pass_progressive,rolling_bound_last6_carries,rolling_bound_last6_carries_progressive,rolling_bound_last6_take_ons_attempted,rolling_bound_last6_take_ons_success,goals_efficiency_ub,assists_efficiency_ub,shots_efficiency_ub,contribution_real_ub,contribution_xgi_ub,contribution_npxgi_ub,contribution_efficiency_xgi_ub,contribution_efficiency_npxgi_ub,goals_efficiency_last3,assists_efficiency_last3,shots_efficiency_last3,contribution_real_last3,contribution_xgi_last3,contribution_npxgi_last3,contribution_efficiency_xgi_last3,contribution_efficiency_npxgi_last3,goals_per_90_ub,assists_per_90_ub,real_contribution_per_90_ub,xg_per_90_ub,npxg_per_90_ub,xa_per_90_ub,xgi_per_90_ub,npxgi_per_90_ub,shots_per_90_ub,shots_on_target_per_90_ub,shot_creating_actions_per_90_ub,goal_creating_actions_per_90_ub,goals_per_90_last3,assists_per_90_last3,real_contribution_per_90_last3,xg_per_90_last3,npxg_per_90_last3,xa_per_90_last3,xgi_per_90_last3,npxgi_per_90_last3,shots_per_90_last3,shots_on_target_per_90_last3,shot_creating_actions_per_90_last3,goal_creating_actions_per_90_last3,team_goals_scored,team_goals_conceded,rolling_goals_scored_last3,rolling_goals_conceded_last3,rolling_goals_scored_last6,rolling_goals_conceded_last6,rolling_goals_scored_ub,rolling_goals_conceded_ub
count,13137.000000,13137.000000,13137.000000,13137.000000,13137.000000,13137.00000,13137.000000,13120.000000,13120.000000,13137.000000,13120.000000,13137.000000,13120.000000,13120.000000,13137.000000,13137.000000,13137.000000,13137.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13136.000000,1.311900e+04,1.311900e+04,1.311900e+04,13136.000000,13136.000000,13119.000000,13136.000000,13136.000000,13119.000000,13136.000000,13136.

In [218]:
nulls = num_df.isna().sum() / num_df.shape[0]
nulls[nulls > 0]

rolling_ub_np_expected_goals               0.001294
rolling_ub_expected_goals                  0.001294
rolling_ub_expected_assists                0.001294
rolling_ub_shots                           0.001294
rolling_ub_shots_on_target                 0.001294
                                             ...   
rolling_bound_last6_pass_progressive       0.001370
rolling_bound_last6_carries                0.001370
rolling_bound_last6_carries_progressive    0.001370
rolling_bound_last6_take_ons_attempted     0.001370
rolling_bound_last6_take_ons_success       0.001370
Length: 68, dtype: float64

In [219]:
cols_to_drop.extend(['player_id', 'game_week', 'team_id'])

#### Categorical Columns

In [220]:
cat_df = df.select_dtypes(include= "object")
cat_df.head()

,season
0,2022-23
1,2022-23
2,2022-23
3,2022-23
4,2022-23


In [221]:
cols_to_drop.extend(list(cat_df.columns))

### Preprocessing

In [222]:
unique_fixtures = df['fix_id'].unique().tolist()

In [223]:
df

,player_id,fix_id,season,game_week,team_id,team_strength,opponent_strength,rolling_ub_minutes,rolling_ub_np_expected_goals,rolling_ub_expected_goals,rolling_ub_goals,rolling_ub_expected_assists,rolling_ub_assists,rolling_ub_shots,rolling_ub_shots_on_target,rolling_ub_pk_made,rolling_ub_pk_attempt,rolling_ub_yellow_cards,rolling_ub_red_cards,rolling_ub_touches,rolling_ub_tackles,rolling_ub_interceptions,rolling_ub_blocks,rolling_ub_shot_creating_actions,rolling_ub_goal_creating_actions,rolling_ub_pass_attempted,rolling_ub_pass_completed,rolling_ub_pass_progressive,rolling_ub_carries,rolling_ub_carries_progressive,rolling_ub_take_ons_attempted,rolling_ub_take_ons_success,rolling_bound_last3_minutes,rolling_bound_last3_expected_goals,rolling_bound_last3_np_expected_goals,rolling_bound_last3_expected_assists,rolling_bound_last3_goals,rolling_bound_last3_assists,rolling_bound_last3_shots,rolling_bound_last3_pk_made,rolling_bound_last3_pk_attempt,rolling_bound_last3_shots_on_target,rolling_bound_last3_yellow_cards,rolling_bound_last3_red_cards,rolling_bound_last3_touches,rolling_bound_last3_tackles,rolling_bound_last3_interceptions,rolling_bound_last3_blocks,rolling_bound_last3_shot_creating_actions,rolling_bound_last3_goal_creating_actions,rolling_bound_last3_pass_completed,rolling_bound_last3_pass_attempted,rolling_bound_last3_pass_progressive,rolling_bound_last3_carries,rolling_bound_last3_carries_progressive,rolling_bound_last3_take_ons_attempted,rolling_bound_last3_take_ons_success,rolling_bound_last6_minutes,rolling_bound_last6_expected_goals,rolling_bound_last6_np_expected_goals,rolling_bound_last6_expected_assists,rolling_bound_last6_goals,rolling_bound_last6_assists,rolling_bound_last6_shots,rolling_bound_last6_pk_made,rolling_bound_last6_pk_attempt,rolling_bound_last6_shots_on_target,rolling_bound_last6_yellow_cards,rolling_bound_last6_red_cards,rolling_bound_last6_touches,rolling_bound_last6_tackles,rolling_bound_last6_interceptions,rolling_bound_last6_blocks,rolling_bound_last6_shot_creating_actions,rolling_bound_last6_goal_creating_actions,rolling_bound_last6_pass_completed,rolling_bound_last6_pass_attempted,rolling_bound_last6_pass_progressive,rolling_bound_last6_carries,rolling_bound_last6_carries_progressive,rolling_bound_last6_take_ons_attempted,rolling_bound_last6_take_ons_success,goals_efficiency_ub,assists_efficiency_ub,shots_efficiency_ub,contribution_real_ub,contribution_xgi_ub,contribution_npxgi_ub,contribution_efficiency_xgi_ub,contribution_efficiency_npxgi_ub,goals_efficiency_last3,assists_efficiency_last3,shots_efficiency_last3,contribution_real_last3,contribution_xgi_last3,contribution_npxgi_last3,contribution_efficiency_xgi_last3,contribution_efficiency_npxgi_last3,goals_per_90_ub,assists_per_90_ub,real_contribution_per_90_ub,xg_per_90_ub,npxg_per_90_ub,xa_per_90_ub,xgi_per_90_ub,npxgi_per_90_ub,shots_per_90_ub,shots_on_target_per_90_ub,shot_creating_actions_per_90_ub,goal_creating_actions_per_90_ub,goals_per_90_last3,assists_per_90_last3,real_contribution_per_90_last3,xg_per_90_last3,npxg_per_90_last3,xa_per_90_last3,xgi_per_90_last3,npxgi_per_90_last3,shots_per_90_last3,shots_on_target_per_90_last3,shot_creating_actions_per_90_last3,goal_creating_actions_per_90_last3,team_goals_scored,team_goals_conceded,rolling_goals_scored_last3,rolling_goals_conceded_last3,rolling_goals_scored_last6,rolling_goals_conceded_last6,rolling_goals_scored_ub,rolling_goals_conceded_ub
0,1005,15,2022-23,2,7,2330,2520,90,NaN,NaN,0,NaN,0,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0.000000,0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0

In [224]:
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop
        
    def fit(self, X, y= None):
        return self
    
    def transform(self, X):
        X = X.copy()
        return X.drop(columns= self.columns_to_drop, errors= "ignore")
    
    def get_feature_names_out(self, names= None):
        return
            
class SkewNormalizationTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, skew_threshold=1.0, method='auto', verbose=False):
        self.skew_threshold = skew_threshold
        self.method = method
        self.verbose = verbose
        self.transformations_ = {}
    
    def _determine_transformation(self, series):
        skew = stats.skew(series.dropna())
        
        if self.method == 'log':
            return 'log'
        elif self.method == 'exp':
            return 'exp'
        elif self.method == 'yeo-johnson':
            return 'yeo-johnson'
        
        if abs(skew) > self.skew_threshold:
            if skew > 0:
                return 'log'
            else:
                return 'exp'
        return None
    
    def _create_transformer(self, transformation):
        if transformation == 'log':
            return FunctionTransformer(np.log1p)
        elif transformation == 'exp':
            return FunctionTransformer(np.expm1)
        elif transformation == 'yeo-johnson':
            return PowerTransformer(method='yeo-johnson')
        return None
    
    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        
        self.transformations_ = {}
        for col in X.columns:
            if not np.issubdtype(X[col].dtype, np.number):
                continue
            
            transformation = self._determine_transformation(X[col])
            
            # if transformation:
            transformer = self._create_transformer(transformation)
            
            if self.verbose:
                print(f"Column {col}: Skew = {stats.skew(X[col].dropna()):.2f}, "
                        f"Transformation = {transformation}")
            
            self.transformations_[col] = transformer
        
        return self
    
    def transform(self, X):
        X = pd.DataFrame(X).copy()
        
        for col, transformer in self.transformations_.items():
            if transformer != None:
                X[col] = transformer.fit_transform(X[[col]])
        
        return X
    
    def get_feature_names_out(self, input_features=None):
        return list(self.transformations_.keys())
    
num_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy= "constant", fill_value= 0)),
    ("skew", SkewNormalizationTransformer()),
    ("scaler", StandardScaler())
])

cat_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy= "most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown= "ignore", sparse_output= False))
])

column_transformer = ColumnTransformer([
    ("num", num_transformer, [col for col in df.select_dtypes(include= np.number) if col != 'fix_id' and col not in cols_to_drop]),
    ("cat", cat_transformer, make_column_selector(dtype_include= "object"))
], remainder= "passthrough")

col_dropper = ColumnDropper(cols_to_drop)
preprocessor = Pipeline([
    ("dropper", col_dropper),
    ("preprocessing", column_transformer)
])

X_preprocessed = pd.DataFrame(preprocessor.fit_transform(df), columns= preprocessor.get_feature_names_out())

outlier_detection = IsolationForest(random_state= 42)
outlier_pred = outlier_detection.fit_predict(X_preprocessed)
X_preprocessed = X_preprocessed[outlier_pred == 1].reset_index(drop= True)
labels = labels[outlier_pred == 1].reset_index(drop= True)

y_goals = labels["label_goals"]
y_assists = labels["label_assists"]

## Modeling

In [225]:
class CustomTimeSeriesCV(BaseCrossValidator):
    def __init__(self, unique_fixtures, folds=4):
        if not isinstance(unique_fixtures, list):
            raise TypeError("unique_fixtures must be a list")
        
        if len(unique_fixtures) == 0:
            raise ValueError("unique_fixtures cannot be empty")
        
        if folds <= 0:
            raise ValueError("Number of folds must be positive")
        
        try:
            self.fix_per_fold = max(1, len(unique_fixtures) // (folds + 1))
        except ZeroDivisionError:
            raise ValueError("Not enough unique fixtures for the specified number of folds")
        
        self.unique_fixtures = unique_fixtures
        self.folds = folds

    def __repr__(self):
        return (f"CustomTimeSeriesCV(n_fixtures={len(self.unique_fixtures)}, "
                f"folds={self.folds}, "
                f"fixtures_per_fold={self.fix_per_fold})")

    def __str__(self):
        return (f"Time Series Cross-Validator\n"
                f"Total Fixtures: {len(self.unique_fixtures)}\n"
                f"Number of Folds: {self.folds}\n"
                f"Fixtures per Fold: {self.fix_per_fold}")

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.folds

    def split(self, X, y=None, groups=None):
        fixture_id_col = 'remainder__fix_id'
        unique_fixt = self.unique_fixtures

        for fold in range(1, self.folds + 1):
            train_end = fold * self.fix_per_fold
            test_start = train_end
            test_end = test_start + self.fix_per_fold

            if test_end > len(unique_fixt):
                test_end = len(unique_fixt)
            
            train_fixtures = unique_fixt[:train_end]
            test_fixtures = unique_fixt[test_start:test_end]

            train_idx = X[X[fixture_id_col].isin(train_fixtures)].index.values
            test_idx = X[X[fixture_id_col].isin(test_fixtures)].index.values

            yield (train_idx, test_idx)

def output1_rmse(y_true, y_pred):
    y_true = y_true[:, 0] if isinstance(y_true, np.ndarray) else y_true.iloc[:, 0].to_numpy()
    y_pred = y_pred[:, 0] if isinstance(y_pred, np.ndarray) else y_pred.iloc[:, 0].to_numpy()
    return np.sqrt(mean_squared_error(y_true, y_pred))

def output2_rmse(y_true, y_pred):
    y_true = y_true[:, 1] if isinstance(y_true, np.ndarray) else y_true.iloc[:, 1].to_numpy()
    y_pred = y_pred[:, 1] if isinstance(y_pred, np.ndarray) else y_pred.iloc[:, 1].to_numpy()
    return np.sqrt(mean_squared_error(y_true, y_pred))

def output1_mae(y_true, y_pred):
    y_true = y_true[:, 0] if isinstance(y_true, np.ndarray) else y_true.iloc[:, 0].to_numpy()
    y_pred = y_pred[:, 0] if isinstance(y_pred, np.ndarray) else y_pred.iloc[:, 0].to_numpy()
    return mean_absolute_error(y_true, y_pred)

def output2_mae(y_true, y_pred):
    y_true = y_true[:, 1] if isinstance(y_true, np.ndarray) else y_true.iloc[:, 1].to_numpy()
    y_pred = y_pred[:, 1] if isinstance(y_pred, np.ndarray) else y_pred.iloc[:, 1].to_numpy()
    return mean_absolute_error(y_true, y_pred)



def train_models(model_pipelines, param_grids, X_train, y_train, n_iter, folds=10, refit="output1_rmse", n_jobs=-1):
    scorers = {
        'output1_rmse': make_scorer(output1_rmse, greater_is_better=False),
        'output1_mae': make_scorer(output1_mae, greater_is_better=False),
        'output2_rmse': make_scorer(output2_rmse, greater_is_better=False),
        'output2_mae': make_scorer(output2_mae, greater_is_better=False),
    }
    
    unique_fixtures = X_train["remainder__fix_id"].unique().tolist()
    cv = CustomTimeSeriesCV(unique_fixtures, folds=folds)
    grids = {}
    
    for model_name, pipeline in model_pipelines.items():
        print(f"Training and tuning {model_name}")
        
        grids[model_name] = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=param_grids[model_name],
            n_iter=n_iter,
            cv=cv,
            scoring=scorers,
            verbose=3,
            n_jobs=n_jobs,
            random_state=42,
            refit=refit,
        )
        
        grids[model_name].fit(X_train, y_train)
        
        if isinstance(scorers, dict) or isinstance(scorers, list):
            print(f"Best parameters for {model_name} based on '{refit}': {grids[model_name].best_params_}")
            print(f"Best Score for {model_name} based on '{refit}': {grids[model_name].best_score_}\n")
        else:
            print(f"Best parameters for {model_name}: {grids[model_name].best_params_}")
            print(f"Best Score for {model_name}: {grids[model_name].best_score_}\n")
    
    return grids


def combine_grid_results(grids, sort_column=None):
    dfs = []
    for name, grid in grids.items():
        df = pd.DataFrame(grid.cv_results_)
        df["model"] = name
        dfs.append(df)
    
    if sort_column != None:
        combined_df = pd.concat(dfs).sort_values(sort_column, ascending=False).reset_index(drop=True)
    else:
        combined_df = pd.concat(dfs).reset_index(drop= True)
    return combined_df

In [226]:
model_num = 10

last_10_perc = len(X_preprocessed["remainder__fix_id"].unique()) // 10
test_fixtures = (X_preprocessed["remainder__fix_id"].tolist()[-last_10_perc:])

train_idx = X_preprocessed[~X_preprocessed["remainder__fix_id"].isin(test_fixtures)].index.values
test_idx = X_preprocessed[X_preprocessed["remainder__fix_id"].isin(test_fixtures)].index.values

X_train = X_preprocessed.iloc[train_idx]
y_train = labels.iloc[train_idx]

X_test = X_preprocessed.iloc[test_idx]
y_test = labels.iloc[test_idx]


model_pipelines = {
    "Linear Regression": Pipeline([
        # ("poly", PolynomialFeatures()),
        ("lr", MultiOutputRegressor(LinearRegression(), n_jobs= -1))
    ]),
    
    "Ridge": Pipeline([
        ("poly", PolynomialFeatures()),
        ("ridge", Ridge(random_state= 42))
    ]),
    
    "Lasso": Pipeline([
        ("poly", PolynomialFeatures()),
        ("lasso", Lasso(random_state= 42))
    ]),

    "Elastic Net": Pipeline([
        ("poly", PolynomialFeatures()),
        ("e_net", ElasticNet(random_state= 42))
    ]),
    
    # "SVR": Pipeline([
    #     ("poly", PolynomialFeatures()),
    #     ("SVC", SVR())
    # ]),

    **{f"Decision Tree {i}": Pipeline(
        [("dt", DecisionTreeRegressor(random_state=42, max_features= "sqrt"))]) for i in range(1, model_num + 1)}
}

param_grids = {
    "Linear Regression": {
        # "poly__degree": randint(1, 2),
    },

    "Ridge": {
        "poly__degree": randint(1, 2),
        "ridge__alpha": uniform(0.01, 10),
    },

    "Lasso": {
        "poly__degree": randint(1, 2),
        "lasso__alpha": uniform(0.01, 1),
    },

    "Elastic Net": {
        "poly__degree": randint(1, 2),
        "e_net__alpha": uniform(0.01, 1),
        "e_net__l1_ratio": uniform(0.01, 1),
    },

    "SVR": {
        "poly__degree": randint(1, 2),
        "SVC__C": uniform(0.1, 10),
        "SVC__epsilon": uniform(0.01, 1),
        "SVC__kernel": ["linear", "poly", "rbf"],
        "SVC__degree": randint(2, 5),
        "SVC__gamma": uniform(0.01, 1),
    },

    **{f"Decision Tree {i}": {
            "dt__max_depth": randint(1, 10 * i),
            "dt__min_samples_split": randint(2 * i, 10 * i),
            "dt__min_samples_leaf": randint(1, i+1),
        }
        for i in range(1, model_num + 1)
    },
}

# scoring = {
#     "neg_root_mean_squared_error": "neg_root_mean_squared_error",
#     "neg_mean_absolute_error": "neg_mean_absolute_error"
# }

base_model_grids = train_models(model_pipelines, param_grids, X_train, y_train, n_iter=10, folds=5)
base_results = combine_grid_results(base_model_grids)

Training and tuning Linear Regression
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/samiadam/miniconda3/envs/fpl-project/lib/python3.10/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
python(8095) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8096) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8097) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8098) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8099) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8100) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8101) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8102) MallocStackLogging: can't turn off malloc

[CV 1/5] END  output1_mae: (test=-0.221) output1_rmse: (test=-0.344) output2_mae: (test=-0.147) output2_rmse: (test=-0.259) total time=   0.1s
[CV 2/5] END  output1_mae: (test=-0.168) output1_rmse: (test=-0.305) output2_mae: (test=-0.118) output2_rmse: (test=-0.252) total time=   0.1s
[CV 3/5] END  output1_mae: (test=-0.203) output1_rmse: (test=-0.629) output2_mae: (test=-0.141) output2_rmse: (test=-0.352) total time=   0.2s
[CV 4/5] END  output1_mae: (test=-0.172) output1_rmse: (test=-0.294) output2_mae: (test=-0.145) output2_rmse: (test=-0.274) total time=   0.3s
[CV 5/5] END  output1_mae: (test=-0.152) output1_rmse: (test=-0.294) output2_mae: (test=-0.126) output2_rmse: (test=-0.252) total time=   0.3s
Best parameters for Linear Regression based on 'output1_rmse': {}
Best Score for Linear Regression based on 'output1_rmse': -0.3733535678650357

Training and tuning Ridge
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END poly__degree=1, ridge__alpha=3.755401188

In [227]:
base_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_output1_rmse,split1_test_output1_rmse,split2_test_output1_rmse,split3_test_output1_rmse,split4_test_output1_rmse,mean_test_output1_rmse,std_test_output1_rmse,rank_test_output1_rmse,split0_test_output1_mae,split1_test_output1_mae,split2_test_output1_mae,split3_test_output1_mae,split4_test_output1_mae,mean_test_output1_mae,std_test_output1_mae,rank_test_output1_mae,split0_test_output2_rmse,split1_test_output2_rmse,split2_test_output2_rmse,split3_test_output2_rmse,split4_test_output2_rmse,mean_test_output2_rmse,std_test_output2_rmse,rank_test_output2_rmse,split0_test_output2_mae,split1_test_output2_mae,split2_test_output2_mae,split3_test_output2_mae,split4_test_output2_mae,mean_test_output2_mae,std_test_output2_mae,rank_test_output2_mae,model,param_poly__degree,param_ridge__alpha,param_lasso__alpha,param_e_net__alpha,param_e_net__l1_ratio,param_dt__max_depth,param_dt__min_samples_leaf,param_dt__min_samples_split
0,0.183251,0.076161,0.023396,0.003944,{},-0.344315,-0.304983,-0.629454,-0.293747,-0.294268,-0.373354,0.129381,1,-0.220658,-0.167931,-0.203306,-0.171752,-0.152151,-0.183159,0.025042,1,-0.258517,-0.252149,-0.352397,-0.273729,-0.251849,-0.277728,0.038170,1,-0.146963,-0.118303,-0.140856,-0.145490,-0.125841,-0.135491,0.011392,1,Linear Regression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.063055,0.020426,0.010719,0.005190,"{'poly__degree': 1, 'ridge__alpha': 3.75540118...",-0.303620,-0.302440,-0.317721,-0.293124,-0.294052,-0.302191,0.008851,7,-0.194391,-0.164401,-0.190132,-0.170965,-0.151927,-0.174363,0.015899,7,-0.243009,-0.252046,-0.276041,-0.273098,-0.251759,-0.259191,0.013004,7,-0.134472,-0.116985,-0.135126,-0.143737,-0.126183,-0.131301,0.009061,7,Ridge,1.0,3.755401,NaN,NaN,NaN,NaN,NaN,NaN
2,0.050631,0.020251,0.011716,0.010132,"{'poly__degree': 1, 'ridge__alpha': 9.51714306...",-0.294374,-0.301544,-0.317158,-0.292813,-0.293969,-0.299972,0.009126,1,-0.186000,-0.162573,-0.189384,-0.170583,-0.151632,-0.172034,0.014170,1,-0.236671,-0.252104,-0.275781,-0.272960,-0.251708,-0.257845,0.014623,1,-0.129901,-0.116160,-0.134788,-0.143181,-0.126220,-0.130050,0.008972,1,Ridge,1.0,9.517143,NaN,NaN,NaN,NaN,NaN,NaN
3,0.033915,0.013139,0.011363,0.003042,"{'poly__degree': 1, 'ridge__alpha': 7.32993941...",-0.297156,-0.301811,-0.317326,-0.292906,-0.293990,-0.300638,0.008899,3,-0.188525,-0.163132,-0.189641,-0.170711,-0.151732,-0.172748,0.014646,3,-0.238642,-0.252088,-0.275837,-0.273004,-0.251720,-0.258258,0.014086,3,-0.131362,-0.116407,-0.134876,-0.143338,-0.126210,-0.130439,0.008963,3,Ridge,1.0,7.329939,NaN,NaN,NaN,NaN,NaN,NaN
4,0.043022,0.022942,0.021844,0.017448,"{'poly__degree': 1, 'ridge__alpha': 5.99658484...",-0.299217,-0.302008,-0.317451,-0.292974,-0.294008,-0.301132,0.008810,6,-0.190390,-0.163539,-0.189812,-0.170797,-0.151800,-0.173268,0.015023,6,-0.240067,-0.252074,-0.275884,-0.273035,-0.251730,-0.258558,0.013714,6,-0.132392,-0.116590,-0.134946,-0.143460,-0.126203,-0.130718,0.008978,6,Ridge,1.0,5.996585,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,0.154878,0.056451,0.009757,0.002798,"{'dt__max_depth': 38, 'dt__min_samples_leaf': ...",-0.342867,-0.330900,-0.346055,-0.353466,-0.331684,-0.340994,0.008640,8,-0.203220,-0.147992,-0.163646,-0.195950,-0.155651,-0.173292,0.022151,8,-0.270638,-0.275069,-0.291631,-0.317832,-0.271765,-0.285387,0.017895,9,-0.120580,-0.119575,-0.117209,-0.168276,-0.120408,-0.129209,0.019570,7,Decision Tree 10,NaN,NaN,NaN,NaN,NaN,38.0,2.0,83.0
127,0.120184,0.074834,0.010278,0.007352,"{'dt__max_depth': 60, 'dt__min_samples_leaf': ...",-0.365341,-0.339579,-0.352287,-0.333610,-0.326325,-0.343428,0.013874,9,-0.208520,-0.173865,-0.170101,-0.163086,-0.155093,-0.174133,0.018349,9,-0.243958,-0.263940,-0.296044,-0.298569,-0.274244,-0.275351,0.020417,5,-0.116977,-0.115269,-0.123241,-0.145

In [240]:
ensemble_model_num = 10
base_estimators = [(name, model.best_estimator_) for name, model in base_model_grids.items()]
ensemble_pipelines = {
    # "VotingClassifier": Pipeline([
    #     ("voting", MultiOutputRegressor(VotingRegressor(estimators=base_estimators, n_jobs=-1)))
    # ]),
    
    # "StackingClassifier": Pipeline([
    #     ("stacking", StackingRegressor(estimators=base_estimators, n_jobs=-1, final_estimator= ElasticNet()))
    # ]),
    
    **{f"Bagging_DecisionTree_{i}": Pipeline([
        ("bc", BaggingRegressor(DecisionTreeRegressor(random_state=42), random_state=42))
        ])
    for i in range(1, ensemble_model_num)},

    **{f"XGBoost_{i}": XGBRegressor(random_state=42)
    for i in range(1, ensemble_model_num + 1)},
    
    **{f"RandomForest_{i}": Pipeline([
        ("rf", RandomForestRegressor(random_state=42, max_features= "sqrt"))
        ]) 
    for i in range(1, ensemble_model_num + 1)}
}

ensemble_param_grids = {
    "VotingClassifier": {
        "voting__estimator__weights": [[1] * len(base_estimators), 
                            [2] * len(base_estimators), 
                            [i for i in range(1, len(base_estimators) + 1)]],
    },

    "StackingClassifier": {
        "stacking__final_estimator__alpha": uniform(0.1, 20),
        # "stacking__final_estimator__poly__degree": randint(1, 2),
        "stacking__final_estimator__l1_ratio": uniform(0.01, 1),
    },

    **{
        f"Bagging_DecisionTree_{i}": {
            "bc__n_estimators": randint(10, 100 * i),
            "bc__max_samples": uniform(0.1, 0.1 + (1 - 0.1) / i),
            "bc__max_features": uniform(0.1, 0.1 + (1 - 0.1) / i),
            "bc__bootstrap": [True, False],
            "bc__bootstrap_features": [True, False]
        }
        for i in range(1, ensemble_model_num + 1)
    },

    **{
        f"XGBoost_{i}": {
            "max_depth": randint(3 , 10 * i),
            "learning_rate": uniform(0.01, 0.2 / i),
            "n_estimators": randint(50, 200 * i),
            "subsample": uniform(0.1, 0.1 + (1 - 0.1) / i),
            "colsample_bytree": uniform(0.1, 0.1 + (1 - 0.1) / i),
        }
        for i in range(1, ensemble_model_num + 1)
    },

    **{
        f"RandomForest_{i}": {
            "rf__n_estimators": randint(100, 300 * i), 
            "rf__max_depth": randint(5 * i, 20 * i),
            "rf__min_samples_split": randint(2 * i, 10 * i),
            "rf__min_samples_leaf": randint(1, 5 * i),
            "rf__bootstrap": [True, False],
        }
        for i in range(1, ensemble_model_num + 1)
    }
}

ensemble_model_grids = train_models(ensemble_pipelines, ensemble_param_grids, X_train, y_train, 10, 5)
ensemble_results = combine_grid_results(ensemble_model_grids)

Training and tuning Bagging_DecisionTree_1
Fitting 5 folds for each of 10 candidates, totalling 50 fits


python(23297) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23298) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23299) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23300) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23301) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23302) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23303) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23304) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[CV 2/5] END bc__bootstrap=True, bc__bootstrap_features=False, bc__max_features=1.0507143064099163, bc__max_samples=0.8319939418114051, bc__n_estimators=70; output1_mae: (test=nan) output1_rmse: (test=nan) output2_mae: (test=nan) output2_rmse: (test=nan) total time=   0.0s
[CV 3/5] END bc__bootstrap=True, bc__bootstrap_features=False, bc__max_features=1.0507143064099163, bc__max_samples=0.8319939418114051, bc__n_estimators=70; output1_mae: (test=nan) output1_rmse: (test=nan) output2_mae: (test=nan) output2_rmse: (test=nan) total time=   0.0s
[CV 1/5] END bc__bootstrap=True, bc__bootstrap_features=False, bc__max_features=1.0507143064099163, bc__max_samples=0.8319939418114051, bc__n_estimators=70; output1_mae: (test=nan) output1_rmse: (test=nan) output2_mae: (test=nan) output2_rmse: (test=nan) total time=   0.0s
[CV 5/5] END bc__bootstrap=True, bc__bootstrap_features=False, bc__max_features=1.0507143064099163, bc__max_samples=0.8319939418114051, bc__n_estimators=70; output1_mae: (test=na

/Users/samiadam/miniconda3/envs/fpl-project/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/samiadam/miniconda3/envs/fpl-project/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/samiadam/miniconda3/envs/fpl-project/lib/python3.10/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/samiadam/miniconda3/envs/fpl-project/lib/python3.10/site-packages

Best parameters for Bagging_DecisionTree_1 based on 'output1_rmse': {'bc__bootstrap': True, 'bc__bootstrap_features': False, 'bc__max_features': 0.12306242504141576, 'bc__max_samples': 0.6247746602583891, 'bc__n_estimators': 51}
Best Score for Bagging_DecisionTree_1 based on 'output1_rmse': -0.3114683685173281

Training and tuning Bagging_DecisionTree_2
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END bc__bootstrap=True, bc__bootstrap_features=True, bc__max_features=0.34520801406947516, bc__max_samples=0.1549862036999016, bc__n_estimators=97; output1_mae: (test=-0.230) output1_rmse: (test=-0.307) output2_mae: (test=-0.157) output2_rmse: (test=-0.238) total time=   1.0s
[CV 2/5] END bc__bootstrap=True, bc__bootstrap_features=True, bc__max_features=0.34520801406947516, bc__max_samples=0.1549862036999016, bc__n_estimators=97; output1_mae: (test=-0.202) output1_rmse: (test=-0.316) output2_mae: (test=-0.145) output2_rmse: (test=-0.257) total time=   1.8s
[CV 3/5] EN

/Users/samiadam/miniconda3/envs/fpl-project/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/samiadam/miniconda3/envs/fpl-project/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/samiadam/miniconda3/envs/fpl-project/lib/python3.10/site-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/Users/samiadam/miniconda3/envs/fpl-project/lib/python3.10/site-packages/xgboost/sklearn.py", lin

Best parameters for XGBoost_1 based on 'output1_rmse': {'colsample_bytree': 0.7116531604882809, 'learning_rate': 0.011413261043943482, 'max_depth': 3, 'n_estimators': 98, 'subsample': 0.6247746602583891}
Best Score for XGBoost_1 based on 'output1_rmse': -0.29281072637674327

Training and tuning XGBoost_2
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END colsample_bytree=0.18581025224334008, learning_rate=0.025599452033620268, max_depth=13, n_estimators=137, subsample=0.283539736126462; output1_mae: (test=-0.218) output1_rmse: (test=-0.303) output2_mae: (test=-0.148) output2_rmse: (test=-0.240) total time=   6.4s
[CV 1/5] END colsample_bytree=0.3059970653660494, learning_rate=0.10507143064099161, max_depth=13, n_estimators=121, subsample=0.4292621663083701; output1_mae: (test=-0.251) output1_rmse: (test=-0.352) output2_mae: (test=-0.200) output2_rmse: (test=-0.293) total time=  14.3s
[CV 2/5] END colsample_bytree=0.18581025224334008, learning_rate=0.0255994520336

KeyboardInterrupt: 

In [232]:
ensemble_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf__bootstrap,param_rf__max_depth,param_rf__min_samples_leaf,param_rf__min_samples_split,param_rf__n_estimators,params,split0_test_output1_rmse,split1_test_output1_rmse,split2_test_output1_rmse,split3_test_output1_rmse,split4_test_output1_rmse,mean_test_output1_rmse,std_test_output1_rmse,rank_test_output1_rmse,split0_test_output1_mae,split1_test_output1_mae,split2_test_output1_mae,split3_test_output1_mae,split4_test_output1_mae,mean_test_output1_mae,std_test_output1_mae,rank_test_output1_mae,split0_test_output2_rmse,split1_test_output2_rmse,split2_test_output2_rmse,split3_test_output2_rmse,split4_test_output2_rmse,mean_test_output2_rmse,std_test_output2_rmse,rank_test_output2_rmse,split0_test_output2_mae,split1_test_output2_mae,split2_test_output2_mae,split3_test_output2_mae,split4_test_output2_mae,mean_test_output2_mae,std_test_output2_mae,rank_test_output2_mae,model
0,10.790712,5.921968,0.314331,0.242007,True,8,1,8,206,"{'rf__bootstrap': True, 'rf__max_depth': 8, 'r...",-0.289298,-0.302435,-0.319823,-0.297112,-0.292975,-0.300329,0.010682,7,-0.198508,-0.162347,-0.173356,-0.178515,-0.149459,-0.172437,0.016415,4,-0.226211,-0.251648,-0.277273,-0.273249,-0.250282,-0.255733,0.018378,5,-0.130457,-0.121454,-0.130142,-0.145066,-0.122893,-0.130003,0.008376,4,RandomForest_1
1,25.821606,10.040684,0.220999,0.135276,False,17,1,8,221,"{'rf__bootstrap': False, 'rf__max_depth': 17, ...",-0.313596,-0.309073,-0.319884,-0.306565,-0.293747,-0.308573,0.008686,10,-0.228426,-0.184281,-0.192096,-0.196757,-0.164743,-0.193260,0.020708,10,-0.233032,-0.256250,-0.279762,-0.278558,-0.254248,-0.260370,0.017371,10,-0.143439,-0.139684,-0.154654,-0.163718,-0.138114,-0.147922,0.009788,10,RandomForest_1
2,9.979763,4.373586,0.162154,0.102893,True,11,3,4,187,"{'rf__bootstrap': True, 'rf__max_depth': 11, '...",-0.288212,-0.301517,-0.320122,-0.296318,-0.291604,-0.299554,0.011218,3,-0.199985,-0.166264,-0.175288,-0.178009,-0.152018,-0.174313,0.015719,5,-0.223704,-0.251704,-0.277382,-0.272835,-0.251217,-0.255368,0.019095,4,-0.125406,-0.122903,-0.139888,-0.145116,-0.127130,-0.132089,0.008766,6,RandomForest_1
3,8.714122,4.129330,0.066019,0.028970,True,8,4,9,230,"{'rf__bootstrap': True, 'rf__max_depth': 8, 'r...",-0.280390,-0.300430,-0.319091,-0.293520,-0.291496,-0.296986,0.012791,2,-0.184664,-0.159629,-0.169728,-0.168798,-0.148124,-0.166189,0.012083,2,-0.222081,-0.250245,-0.276618,-0.272461,-0.250504,-0.254382,0.019477,2,-0.121269,-0.118574,-0.129643,-0.138397,-0.122515,-0.126080,0.007164,2,RandomForest_1
4,8.844819,2.805459,0.052489,0.012754,False,9,2,9,257,"{'rf__bootstrap': False, 'rf__max_depth': 9, '...",-0.289355,-0.301791,-0.318787,-0.295580,-0.292397,-0.299582,0.010449,5,-0.197780,-0.162626,-0.175371,-0.173010,-0.149513,-0.171660,0.015937,3,-0.223495,-0.251842,-0.277227,-0.272570,-0.250070,-0.255041,0.019129,3,-0.122476,-0.121269,-0.133517,-0.139192,-0.123407,-0.127972,0.007108,3,RandomForest_1
5,2.366542,1.055815,0.022957,0.004688,False,6,4,5,120,"{'rf__bootstrap': False, 'rf__max_depth': 6, '...",-0.277785,-0.299983,-0.319730,-0.292626,-0.294204,-0.296865,0.013577,1,-0.179440,-0.154392,-0.167432,-0.163090,-0.147494,-0.162369,0.010977,1,-0.221230,-0.250591,-0.277729,-0.272127,-0.250124,-0.254360,0.019958,1,-0.116768,-0.114463,-0.123173,-0.132853,-0.120025,-0.121456,0.006415,1,RandomForest_1
6,6.185487,3.243997,0.055053,0.009146,True,16,2,7,188,"{'rf__bootstrap': True, 'rf__max_depth': 16, '...",-0.298391,-0.303461,-0.319267,-0.301455,-0.292000,-0.302915,0.009049,9,-0.215095,-0.175859,-0.183875,-0.191972,-0.158486,-0.185058,0.018666,9,-0.228024,-0.252716,-0.278533,-0.273504,-0.253743,-0.257304,0.017912,9,-0.136444,-0.133317,-0.150462,-0.153019,-0.134146,-0.141478,0.008480,9,RandomForest_1
7,7.889487,3.974906,0.079998,0.037923,True,15,3,8,269,"{'rf__bootstrap': True, 'rf__max_depth': 15, '...",-0.288164,-0.302457,-0.318777,-0.297273,-0.291194,-0.299573,0.010796,4,-0.201507,-0.172538,-0.178605,-0.180

In [239]:
for i in range(1, 20):
    print((0.1, 0.1 + (1 - 0.1) / i))

(0.1, 1.0)
(0.1, 0.55)
(0.1, 0.4)
(0.1, 0.325)
(0.1, 0.28)
(0.1, 0.25)
(0.1, 0.2285714285714286)
(0.1, 0.21250000000000002)
(0.1, 0.2)
(0.1, 0.19)
(0.1, 0.18181818181818182)
(0.1, 0.175)
(0.1, 0.16923076923076924)
(0.1, 0.1642857142857143)
(0.1, 0.16)
(0.1, 0.15625)
(0.1, 0.15294117647058825)
(0.1, 0.15000000000000002)
(0.1, 0.1473684210526316)
